## Что можно

Современные алгоритмы позволяют достаточно эффективно решать следующие задачи:
* Проверить является ли данный многочлен SOS
* Для многочлена $p(x_1, \ldots, x_n, c_1, \ldots, c_m)$ *линейного* относительно $c_1, \ldots, c_m$ и *линейной* функции $\ell(c_1, \ldots, c_m)$ найти максимум $\ell(c_1, \ldots, c_m)$ среди всех наборов чисел $c_1, \ldots, c_m$ таких, что $p(x_1, \ldots, x_n, c_1, \ldots, c_m)$ является SOS *относительно* $c_1, \ldots, c_m$.
* Все это ещё можно делать не на всем пространстве, а при наличии условий на $x_1, \ldots, x_n$ (про это завтра).

Одной из библиотек, которая умеет делать такие вещи является SumOfSquares.jl на языке Julia.

In [1]:
using SumOfSquares
using DynamicPolynomials
using Mosek
using MosekTools

┌ Info: Precompiling SumOfSquares [4b9e565b-77fc-50a5-a571-1244f986bda1]
└ @ Base loading.jl:1260
┌ Info: Precompiling MosekTools [1ec41992-ff65-5c91-ac43-2df89e9693a4]
└ @ Base loading.jl:1260


## Проверка SOS

**Пример 1.** Неравенство о средних для двух чисел $x^2 + y^2 - 2 xy \geqslant 0$

In [ ]:
@polyvar x y
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)
con_ref = @constraint(model, x^2 - 2 * x * y + y^2 >= 0)
optimize!(model)
@show termination_status(model)
round(sos_decomposition(con_ref)[1], digits=3)

**Пример 2.** Неравенство о средних для трех чисел $x^6 + y^6 + z^6 - 3 x^2 y^2 z^2 \geqslant 0$

In [ ]:
@polyvar x y z
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)
con_ref = @constraint(model, x^6 + y^6 + z^6 - 3 * x^2 * y^2 * z^2 >= 0)
optimize!(model)
@show termination_status(model)
for p = sos_decomposition(con_ref)
    print(round(p, digits=3))
end

**Пример 3.** Многочлен Моцкина $x^4 y^2 + x^2 y^4 - 3 x^2 y^2 + 1$

In [ ]:
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)
moz = x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
@constraint(model, moz >= 0)
optimize!(model)
@show termination_status(model)

**Пример 4.** А теперь умножим на $(x^2 + y^2)$

In [ ]:
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)
moz = x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
con_ref = @constraint(model, moz * (x^2 + y^2) >= 0)
optimize!(model)
@show termination_status(model)
for p = sos_decomposition(con_ref)
    print(round(p, digits=3), "\n")
end

In [ ]:
(-x^3*y - x*y^3 + 2.0*x*y)^2 + (-x^2*y + y)^2 + (x*y^2 - x)^2

In [ ]:
moz * (x^2 + y^2)

## SOS оптимизация

Простешим примером использования SOS оптимизации является поиск нижних оценко на значение многочлена. Например, если мы хотим найти нижнюю оценку на $p(x_1, \ldots, x_n)$, мы можем поискать максимум $c$ такой, что $p(x_1, \ldots, x_n) - c$ является SOS

In [ ]:
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)
@variable(model, c)
moz = x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
@constraint(model, moz - c >= 0)
@objective(model, Max, c)
optimize!(model)
@show termination_status(model)
@show round(value(c), digits = 3)

Так мы выяснили, что многочлен Моцкина всегда $\geqslant -0.655$. Но ведь мы же знаем, что он неотрицателен! Чтобы получить оценку получше, можно прибегнуть к теореме Артина (а точнее Пойа-Резника)

In [ ]:
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)
@variable(model, c)
moz = x^4 * y^2 + x^2 * y^4 - 3 * x^2 * y^2 + 1
@constraint(model, moz * (x^2 + y^2) - c * (x^2 + y^2) >= 0)
@objective(model, Max, c)
optimize!(model)
@show termination_status(model)
@show round(value(c), digits = 3)